In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\training.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,...,Category_Cluster_x,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,0,0,-1.721242,0.094898,-0.733313,0.988754,-0.378194,-0.362793,-0.100989,0.183348,...,0.596018,0.0,0.141625,1.225846,-1.858123,0.166667,0.008288,0.020214,0.000164,0.000004
1,1,1,-1.141648,-0.732741,0.326218,0.898341,-0.377046,-0.356911,-0.100484,-0.207709,...,0.596018,0.0,0.141625,0.113043,-0.287141,0.666667,0.240164,0.033996,0.008053,0.000112
2,2,2,-0.104824,-0.486586,0.469163,0.910707,-0.343066,-0.321594,-0.089758,-0.222435,...,0.596018,0.0,0.141625,0.391244,0.236520,0.000000,9.482668,0.018693,0.173856,0.003405
3,3,3,1.301536,1.291973,-0.315894,0.944730,-0.142867,0.189549,-0.084092,-0.221800,...,-0.298799,0.0,0.141625,0.669444,0.760181,0.333333,49.579964,0.036200,1.790621,0.004178
4,4,4,-0.865923,1.563195,-1.139314,0.892396,-0.354881,-0.301446,-0.006584,0.017232,...,-1.193616,0.0,0.141625,0.113043,-0.287141,0.666667,0.098967,0.041675,0.003743,0.000381


# SKEWNESNI ANIQLASH 

In [3]:
# faqat raqamli ustunlar uchun
skewness = df.select_dtypes(include=['number']).skew()

# Natijani ko‘rish
print("📊 Skewness (egiklik) qiymatlari:\n")
print(skewness)


📊 Skewness (egiklik) qiymatlari:

Unnamed: 0.1          0.000000e+00
Unnamed: 0            0.000000e+00
Video ID              3.347078e-18
Video Title           7.233901e-03
Channel               4.222072e-03
Published Date       -2.672121e+00
Views                 7.030806e+00
Likes                 7.650718e+00
Comments              5.890533e+01
Subscribers           2.001306e+01
Channel Views         2.551080e+01
Country              -3.055750e-01
Region               -8.150671e-02
CategoryID           -8.339317e-01
CategoryName         -2.034283e-02
Cluster               5.532678e+00
CategoryID_enc        7.882981e-03
VideoCluster          1.833848e-01
Channel_Cluster       4.320688e+00
Video_Cluster         2.354969e+00
Country_Cluster_x     1.648645e-01
Category_Cluster_x    6.986700e-02
Year                  0.000000e+00
Month                -6.920496e+00
Day                  -8.059853e-01
DayOfWeek            -3.621841e-01
DayName               6.716106e-02
Views_per_Sub        

In [14]:
import numpy as np
import pandas as pd

# Asl DataFrame ni nusxalab olamiz
df = df.copy()

# 1️⃣ Transformatsiyadan oldingi skewness
before_skew = df.select_dtypes(include=['number']).skew().reset_index()
before_skew.columns = ['Column', 'Before_Skew']

# 2️⃣ Positive va Negative skew ustunlar
positive_cols = [
    'Views', 'Likes', 'Comments', 'Subscribers', 'Channel Views',
    'Cluster', 'Channel_Cluster', 'Video_Cluster',
    'Views_per_Sub', 'Engagement', 'Like_per_Sub', 'Comment_per_Sub'
]

negative_cols = ['Published Date', 'CategoryID', 'Month']

# 3️⃣ Agar datetime bo‘lsa → raqamga o‘tkazamiz
if np.issubdtype(df['Published Date'].dtype, np.datetime64):
    df['Published Date'] = df['Published Date'].astype('int64') // 10**9

# 4️⃣ Positive skew ustunlarga log1p transformatsiya
for col in positive_cols:
    if col in df.columns:
        min_val = df[col].min()
        df[col] = np.log1p(df[col] - min_val + 1)

# 5️⃣ Negative skew ustunlarga kvadrat (yoki exp) transformatsiya
for col in negative_cols:
    if col in df.columns:
        min_val = df[col].min()
        df[col] = (df[col] - min_val + 1) ** 2

# 6️⃣ Transformatsiyadan keyingi skewness
after_skew = df.select_dtypes(include=['number']).skew().reset_index()
after_skew.columns = ['Column', 'After_Skew']

# 7️⃣ Taqqoslash jadvali
compare_skew = pd.merge(before_skew, after_skew, on='Column', how='outer')
compare_skew['Change'] = compare_skew['After_Skew'] - compare_skew['Before_Skew']
compare_skew = compare_skew.sort_values(by='Change')

# 8️⃣ Jadvalni chiqaramiz
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print("📊 Skewness Before vs After:\n")
print(compare_skew)


📊 Skewness Before vs After:

                Column   Before_Skew    After_Skew        Change
8      Comment_per_Sub  4.483139e+01  3.413096e+01 -1.070043e+01
9             Comments  1.270657e+01  8.409148e+00 -4.297422e+00
16        Like_per_Sub  6.316931e+00  4.083229e+00 -2.233701e+00
5        Channel Views  7.229259e+00  5.264421e+00 -1.964838e+00
21         Subscribers  5.597117e+00  4.271818e+00 -1.325300e+00
15          Engagement  6.130045e+00  5.287494e+00 -8.425512e-01
29       Views_per_Sub  1.417450e+00  7.596081e-01 -6.578415e-01
17               Likes  3.684591e+00  3.028245e+00 -6.563463e-01
28               Views  3.565827e+00  2.930305e+00 -6.355215e-01
27       Video_Cluster  2.267517e+00  2.236312e+00 -3.120570e-02
6      Channel_Cluster  4.163608e+00  4.139428e+00 -2.417988e-02
7              Cluster  5.506448e+00  5.500390e+00 -6.057838e-03
11   Country_Cluster_x  1.648645e-01  1.648645e-01  0.000000e+00
4              Channel  4.222072e-03  4.222072e-03  0.000000e

# Models Training Process 

# RandomForestRegressor TRAINING 

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# X va y ajratamiz
X = df.drop('CategoryName', axis=1)
y = df['CategoryName']

# Train / Validation / Test (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)
# (0.25 * 0.8 = 0.2 => 60/20/20)

# Model
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    class_weight='balanced',
    random_state=42
)
rf.fit(X_train, y_train)

# Validation natijalar
y_pred = rf.predict(X_val)

print("✅ Validation Accuracy:", accuracy_score(y_val, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_val, y_pred))
print("\n🧩 Confusion Matrix:\n", confusion_matrix(y_val, y_pred))


ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

# Save RandomForestRegressor Model 

In [ ]:
import joblib
joblib.dump(rf, r"C:\Users\Rasulbek907\Desktop\Final_Project\Models\Simple_Models\RandomForestRegressor.joblib")
print("✅ Model muvaffaqiyatli saqlandi.")

# DecisionTreeRegressor TRAINING 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Model
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

# Baholash
y_pred_dt = dt.predict(X_val)
print("📊 Decision Tree Validation R2:", r2_score(y_val, y_pred_dt))
print("📉 Validation MSE:", mean_squared_error(y_val, y_pred_dt))

# Save DecisionTreeRegressor Model 

In [ ]:
import joblib
joblib.dump(dt, r"C:\Users\Rasulbek907\Desktop\Final_Project\Models\Simple_Models\DecisionTreeRegressor.joblib")
print("✅ Model muvaffaqiyatli saqlandi.")

# Jami: Natijalarni Taqqoslash 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go

# 1. X va y
X = df.drop('life_span', axis=1)
y = df['life_span']

# 2. Train, validation, test bo'lish
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# 3. Modellar
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    r2 = r2_score(y_val, preds)
    mse = mean_squared_error(y_val, preds)
    results.append({'Model': name, 'R2 Score': r2, 'MSE': mse})

# 4. Natijalarni DataFrame ga yig'ish
results_df = pd.DataFrame(results)

# 5. Eng yaxshi R2 topish
best_model_index = results_df['R2 Score'].idxmax()

# 6. Jadvalni chizish (qizil bilan eng yaxshisi)
colors = ['lightgray'] * len(results_df)
colors[best_model_index] = 'crimson'  # eng yaxshi R2 uchun qizil rang

fig = go.Figure(data=[go.Table(
    header=dict(values=list(results_df.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[results_df[col] for col in results_df.columns],
               fill_color=[colors, colors, colors],
               align='left'))
])

fig.update_layout(title='Model Natijalari: R2 Score va MSE')
fig.show()

# Cros Validation Tekshirish Natijalari 

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Model
dt = DecisionTreeRegressor(random_state=42)

# R2 Score bo‘yicha 5-fold cross-validation
r2_scores = cross_val_score(dt, X, y, cv=5, scoring='r2')
print("📊 R2 (5-fold):", r2_scores)
print("📈 R2 O'rtacha:", np.mean(r2_scores))

# Cross-val prediction orqali MSE hisoblash
y_pred_cv = cross_val_predict(dt, X, y, cv=5)
mse = mean_squared_error(y, y_pred_cv)
print("📉 Cross-Validated MSE:", mse)

# Shap Value Bo`yicha Taxliliy Xulosalar 

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

# Modelni o'qitamiz (cross_val uchun emas, alohida fit qilamiz)
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X, y)

# SHAP explainer yaratamiz
explainer = shap.Explainer(dt_model, X)
shap_values = explainer(X)

# Summary plot (eng muhim xususiyatlarni ko‘rsatadi)
shap.summary_plot(shap_values, X)

### ✅ XULOSA

🔍 Yuqoridagi tahliliy natijalardan kelib chiqib, quyidagi xulosani chiqarishimiz mumkin:

📈 **Modelimiz samaradorligini oshirishda aynan o‘zimiz tomonidan yaratilgan quyidagi ustunlar muhim rol o‘ynamoqda**:

- `life_span_boxcox` ⚙️  
- `life_span_log` 📊  
- `life_span_cluster` 🔗  
- `occupation_cluster` 🧠  
- `birth_year_boxcox` 🕰️  
- `death_year_boxcox` ⚰️  

🚀 Ushbu ustunlar **modelimizning eng muhim TOP-10 xususiyatlari** qatoriga kirgan va ularning mavjudligi **aniqlikni sezilarli darajada oshirgan**.

🏅 Shuningdek, `awards` ustuni ham **ahamiyatli omillardan biri** sifatida ajralib turibdi.

---

🎯 **Xulosa qilib aytganda**, **to‘g‘ri xususiyatlar muhandisligi (feature engineering)** model samaradorligining asosiy omillaridan biri bo‘lib xizmat qilmoqda. Bu — muvaffaqiyat sari muhim qadam! 💡📊
